In [1]:
import pandas as pd
from services.jira_service import JiraService  # seu service acima
from datetime import datetime
from services.jira_parser import parse_issues_to_dataframe, parse_issues_time_to_dataframe
import json


jira = JiraService()
issues = jira.get_all_issues_from_project("BL")  # ou outro projeto

df = parse_issues_time_to_dataframe(issues["issues"])
df.to_excel("relatorio_acompanhamento_ti.xlsx", index=False)


🔄 Buscando todas as issues completas do projeto 'BL'...
✅ 429 issues encontradas no projeto 'BL'.


In [2]:

for i, issue in enumerate(issues["issues"]):
    if issue["key"] == "BL-442":
        print(issue)

 






{'expand': 'operations,versionedRepresentations,editmeta,changelog,renderedFields', 'id': '16974', 'self': 'https://tigruposalus.atlassian.net/rest/api/3/issue/16974', 'key': 'BL-442', 'fields': {'statuscategorychangedate': '2025-06-12T15:48:05.234-0300', 'issuetype': {'self': 'https://tigruposalus.atlassian.net/rest/api/3/issuetype/10349', 'id': '10349', 'description': 'Tarefas rastreiam partes de trabalho pequenas e distintas.', 'iconUrl': 'https://tigruposalus.atlassian.net/rest/api/2/universal_avatar/view/type/issuetype/avatar/10318?size=medium', 'name': 'Tarefa', 'subtask': False, 'avatarId': 10318, 'entityId': 'cb763b08-fee4-4b69-82b4-3e92d9cfedce', 'hierarchyLevel': 0}, 'timespent': 6600, 'project': {'self': 'https://tigruposalus.atlassian.net/rest/api/3/project/10085', 'id': '10085', 'key': 'BL', 'name': 'Acompanhamento TI', 'projectTypeKey': 'business', 'simplified': True, 'avatarUrls': {'48x48': 'https://tigruposalus.atlassian.net/rest/api/3/universal_avatar/view/type/project